# Chapter 24: JavaScript and Browser APIs

---

## Introduction

The browser provides a rich set of APIs (Application Programming Interfaces) that allow JavaScript to interact with the browser environment and the user's device. These APIs give you control over timers, the current window, navigation, storage, and even the ability to observe changes in the DOM or element visibility.

In this chapter, you will explore several essential browser APIs:

- **Timers** – scheduling code execution with `setTimeout` and `setInterval`.
- **The `window` object** – controlling the browser window, scrolling, and dialog boxes.
- **The `navigator` object** – detecting browser capabilities, online status, and using the Clipboard API.
- **The `location` object** – working with the current URL, redirecting, and reloading.
- **Web Storage API** – persisting data locally with `localStorage` and `sessionStorage`.
- **Intersection Observer** – efficiently detecting when elements enter or exit the viewport.
- **Mutation Observer** – reacting to changes in the DOM.
- **Resize Observer** – responding to changes in an element's size.

Mastering these APIs will enable you to build more interactive, responsive, and performant web applications.

---

## 24.1 Timers

Timers allow you to schedule the execution of code after a delay or at regular intervals. They are fundamental for animations, debouncing user input, polling, and many other tasks.

### `setTimeout()`

`setTimeout(callback, delay)` executes a function once after a specified number of milliseconds.

```javascript
console.log('Start');
setTimeout(() => {
    console.log('Executed after 2 seconds');
}, 2000);
console.log('End');
// Output: Start, End, (after 2s) Executed after 2 seconds
```

The `delay` is specified in milliseconds (1000 ms = 1 second). If omitted, it defaults to 0, but the callback is still queued asynchronously.

### `setInterval()`

`setInterval(callback, interval)` repeatedly executes a function every `interval` milliseconds.

```javascript
let count = 0;
const intervalId = setInterval(() => {
    console.log(`Tick ${++count}`);
    if (count === 5) {
        clearInterval(intervalId);
    }
}, 1000);
// Logs: Tick 1, Tick 2, ..., Tick 5, then stops.
```

### Clearing Timers

Both `setTimeout` and `setInterval` return a unique numeric ID that can be used to cancel the timer with `clearTimeout(id)` or `clearInterval(id)`.

```javascript
const timeoutId = setTimeout(() => console.log('Wont run'), 5000);
clearTimeout(timeoutId); // cancels it

const intervalId = setInterval(() => console.log('Running'), 1000);
setTimeout(() => clearInterval(intervalId), 5000); // stops after 5 seconds
```

### Debouncing and Throttling

These are patterns built on timers to control how often a function is executed, especially in response to frequent events like scrolling, resizing, or typing.

**Debouncing** ensures that a function is only called after a certain amount of time has passed since the last event. Useful for autocomplete search inputs.

```javascript
function debounce(func, delay) {
    let timeoutId;
    return function (...args) {
        clearTimeout(timeoutId);
        timeoutId = setTimeout(() => func.apply(this, args), delay);
    };
}

const expensiveSearch = debounce((query) => {
    console.log('Searching for:', query);
    // API call
}, 300);

searchInput.addEventListener('input', (e) => expensiveSearch(e.target.value));
```

**Throttling** ensures a function is called at most once every specified interval. Useful for scroll or resize handlers.

```javascript
function throttle(func, limit) {
    let inThrottle;
    return function (...args) {
        if (!inThrottle) {
            func.apply(this, args);
            inThrottle = true;
            setTimeout(() => (inThrottle = false), limit);
        }
    };
}

const handleScroll = throttle(() => {
    console.log('Scrolled at', new Date());
}, 1000);

window.addEventListener('scroll', handleScroll);
```

---

## 24.2 The `window` Object

The `window` object represents the browser window or tab. It is the global object in client‑side JavaScript, meaning all global variables and functions are properties of `window`.

### Window Properties

Important properties include:

- `window.innerWidth`, `window.innerHeight` – viewport dimensions (including scrollbar if present).
- `window.outerWidth`, `window.outerHeight` – the entire browser window size.
- `window.pageXOffset`, `window.pageYOffset` – the amount the document has been scrolled horizontally/vertically (aliases: `scrollX`, `scrollY`).
- `window.screenX`, `window.screenY` – the distance from the left/top edge of the screen to the browser window.

```javascript
console.log(`Viewport: ${window.innerWidth} x ${window.innerHeight}`);
console.log(`Scrolled: ${window.pageYOffset}px`);
```

### Window Dimensions and Resize Events

You can listen to the `resize` event to react to window size changes. Use debouncing or throttling to avoid performance issues.

```javascript
window.addEventListener('resize', () => {
    console.log('New size:', window.innerWidth, window.innerHeight);
});
```

### Scrolling Methods

- `window.scrollTo(x, y)` – scrolls to absolute coordinates.
- `window.scrollBy(x, y)` – scrolls relative to the current position.
- `window.scrollTo(options)` – accepts an object with `top`, `left`, and `behavior: 'smooth'` for smooth scrolling.

```javascript
// Smooth scroll to top
window.scrollTo({ top: 0, behavior: 'smooth' });

// Scroll down by 100px
window.scrollBy(0, 100);
```

### Dialog Boxes

- `alert(message)` – displays a modal dialog with an OK button.
- `confirm(message)` – displays a dialog with OK and Cancel; returns `true` if OK is clicked, otherwise `false`.
- `prompt(message, defaultValue)` – displays a dialog with an input field; returns the entered string or `null` if canceled.

```javascript
const userConfirmed = confirm('Are you sure you want to delete?');
if (userConfirmed) {
    // proceed with deletion
}

const name = prompt('What is your name?', 'Guest');
console.log('Hello,', name || 'Guest');
```

These dialogs block the main thread, so use them sparingly. For custom modal dialogs, consider using HTML and CSS.

---

## 24.3 The `navigator` Object

The `navigator` object contains information about the browser and the device. It is read‑only and provides a wealth of data.

### Browser Information

- `navigator.userAgent` – the browser's user agent string (can be spoofed).
- `navigator.platform` – the operating system platform (e.g., 'Win32', 'MacIntel').
- `navigator.language` – the user's preferred language (e.g., 'en-US').
- `navigator.onLine` – boolean indicating whether the browser is online.

```javascript
if (navigator.onLine) {
    console.log('You are online');
} else {
    console.log('You are offline');
}

window.addEventListener('online', () => console.log('Back online'));
window.addEventListener('offline', () => console.log('Lost connection'));
```

### User Agent

Parsing `userAgent` is tricky and error‑prone. Instead, use feature detection. However, sometimes you need it for analytics:

```javascript
console.log(navigator.userAgent);
// Example: "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 ..."
```

### Clipboard API

The Clipboard API provides asynchronous read and write access to the system clipboard. It requires user permission (triggered by a user action like a click).

**Writing to clipboard:**

```javascript
async function copyText(text) {
    try {
        await navigator.clipboard.writeText(text);
        console.log('Text copied to clipboard');
    } catch (err) {
        console.error('Failed to copy:', err);
    }
}

// Usage (inside a click handler)
copyButton.addEventListener('click', () => copyText('Hello, world!'));
```

**Reading from clipboard:**

```javascript
async function pasteText() {
    try {
        const text = await navigator.clipboard.readText();
        console.log('Pasted content:', text);
    } catch (err) {
        console.error('Failed to read clipboard:', err);
    }
}
```

**Important:** The Clipboard API requires a secure context (HTTPS or localhost) and may prompt the user for permission.

---

## 24.4 The `location` Object

The `location` object is part of the `window` object and represents the current URL. It provides methods to navigate and reload the page.

### URL Components

- `location.href` – the full URL.
- `location.protocol` – e.g., 'http:' or 'https:'.
- `location.host` – hostname + port.
- `location.hostname` – domain name.
- `location.port` – port number.
- `location.pathname` – path after the domain.
- `location.search` – query string (including `?`).
- `location.hash` – fragment identifier (including `#`).

```javascript
console.log(location.href);      // "https://example.com/page?q=test#section"
console.log(location.pathname);   // "/page"
console.log(location.search);     // "?q=test"
console.log(location.hash);       // "#section"
```

### Navigating and Redirecting

- `location.assign(url)` – loads the new document.
- `location.replace(url)` – replaces the current document with a new one (no entry in session history).
- `location.reload()` – reloads the current page (optionally `true` to force reload from server).

```javascript
// Redirect to another page
location.assign('https://google.com');

// Redirect without preserving the current page in history
location.replace('/new-page');

// Reload
location.reload();            // may use cache
location.reload(true);        // force reload from server (deprecated, use cache‑busting instead)
```

You can also modify `location.href` directly: `location.href = 'https://example.com'` is equivalent to `assign()`.

### Parsing Query Strings

You can parse the query string using `URLSearchParams`:

```javascript
const params = new URLSearchParams(location.search);
const q = params.get('q');        // "test"
const hasFilter = params.has('filter');
params.set('page', '2');
// To update the URL without reloading, use history.pushState (see below)
```

---

## 24.5 Web Storage API

Web Storage provides a way to store key‑value pairs locally in the browser. It's simpler than cookies and has a larger storage limit (typically 5‑10 MB per origin).

### `localStorage` vs `sessionStorage`

- **`localStorage`** – persists even after the browser is closed and reopened. Shared across all tabs/windows from the same origin.
- **`sessionStorage`** – persists only for the duration of the page session. Data is cleared when the tab is closed. Not shared between tabs.

### Basic Methods

All methods are synchronous.

```javascript
// Store data
localStorage.setItem('username', 'alice');
sessionStorage.setItem('theme', 'dark');

// Retrieve data
const user = localStorage.getItem('username'); // "alice"
const theme = sessionStorage.getItem('theme'); // "dark"

// Remove a specific item
localStorage.removeItem('username');

// Clear all items
sessionStorage.clear();
```

### Storing Complex Data (JSON)

Storage only supports strings. To store objects or arrays, use `JSON.stringify()` and `JSON.parse()`.

```javascript
const user = { name: 'Bob', age: 30 };
localStorage.setItem('user', JSON.stringify(user));

const retrieved = JSON.parse(localStorage.getItem('user'));
console.log(retrieved.name); // "Bob"
```

### Storage Limits

Quota varies by browser, typically 5‑10 MB per origin. Attempting to exceed the quota throws a `QuotaExceededError`. Always handle potential errors.

```javascript
try {
    localStorage.setItem('largeData', bigString);
} catch (e) {
    if (e.name === 'QuotaExceededError') {
        console.error('Storage limit exceeded');
    }
}
```

### Storage Events

When data in `localStorage` is changed in another tab/window of the same origin, a `storage` event is fired on the `window` object. This allows you to synchronize state across tabs.

```javascript
window.addEventListener('storage', (event) => {
    console.log('Key changed:', event.key);
    console.log('Old value:', event.oldValue);
    console.log('New value:', event.newValue);
    console.log('Storage area:', event.storageArea); // localStorage or sessionStorage
});
```

Note: The `storage` event does **not** fire on the tab that made the change—only on other tabs/windows listening.

---

## 24.6 Intersection Observer

The Intersection Observer API provides a way to asynchronously observe changes in the intersection of a target element with an ancestor element or with the viewport. It is highly efficient and ideal for lazy loading images, implementing infinite scroll, and triggering animations when elements become visible.

### Creating an Observer

```javascript
const observer = new IntersectionObserver((entries, observer) => {
    entries.forEach(entry => {
        if (entry.isIntersecting) {
            console.log('Element is visible:', entry.target);
            // Do something (e.g., load image)
            // Optionally unobserve after it becomes visible
            observer.unobserve(entry.target);
        }
    });
}, {
    root: null,        // use viewport as root (default)
    rootMargin: '0px',  // margin around the root
    threshold: 0.1      // percentage of visibility needed to trigger (0.1 = 10%)
});

// Start observing an element
const target = document.getElementById('lazy-image');
observer.observe(target);
```

### Options

- **`root`** – the element used as the viewport for checking visibility. Defaults to the browser viewport (`null`).
- **`rootMargin`** – a CSS‑like margin string that expands or shrinks the root's bounding box. Can be used to trigger earlier/later.
- **`threshold`** – a number or array of numbers between 0 and 1 indicating at what percentage of the target's visibility the callback should be executed.

### Use Cases

**Lazy loading images:**

```html
<img class="lazy" data-src="real-image.jpg" alt="Lazy">
```

```javascript
const lazyImages = document.querySelectorAll('.lazy');
const observer = new IntersectionObserver((entries, obs) => {
    entries.forEach(entry => {
        if (entry.isIntersecting) {
            const img = entry.target;
            img.src = img.dataset.src;
            img.classList.add('loaded');
            obs.unobserve(img);
        }
    });
}, { rootMargin: '200px' }); // start loading 200px before the image enters the viewport

lazyImages.forEach(img => observer.observe(img));
```

**Infinite scroll:**

```javascript
const sentinel = document.getElementById('load-more-sentinel');
const observer = new IntersectionObserver((entries) => {
    if (entries[0].isIntersecting) {
        loadMoreItems();
    }
}, { threshold: 1.0 }); // trigger when sentinel is fully visible

observer.observe(sentinel);
```

**Triggering animations when an element enters view:**

```javascript
const animatedElements = document.querySelectorAll('.animate-on-scroll');
const observer = new IntersectionObserver((entries) => {
    entries.forEach(entry => {
        if (entry.isIntersecting) {
            entry.target.classList.add('fade-in');
        }
    });
}, { threshold: 0.5 });

animatedElements.forEach(el => observer.observe(el));
```

---

## 24.7 Mutation Observer

The Mutation Observer API lets you watch for changes to the DOM tree: addition or removal of elements, attribute changes, or text content changes. It is a powerful alternative to the deprecated mutation events, providing better performance and control.

### Creating a Mutation Observer

```javascript
const observer = new MutationObserver((mutationsList, observer) => {
    for (const mutation of mutationsList) {
        if (mutation.type === 'childList') {
            console.log('Child nodes added/removed:', mutation);
        } else if (mutation.type === 'attributes') {
            console.log(`Attribute ${mutation.attributeName} changed on`, mutation.target);
        } else if (mutation.type === 'characterData') {
            console.log('Text content changed');
        }
    }
});

// Start observing
observer.observe(targetNode, {
    attributes: true,        // watch for attribute changes
    childList: true,         // watch for child node addition/removal
    subtree: true,           // watch the entire subtree
    characterData: true,     // watch for text content changes
    attributeOldValue: true, // record old attribute value
    characterDataOldValue: true, // record old character data
    attributeFilter: ['class', 'style'] // only watch specific attributes
});

// To stop observing
observer.disconnect();
```

### Use Cases

- **Detecting when a script dynamically adds content** – e.g., a chat application that needs to scroll to the latest message.
- **Reacting to attribute changes** – e.g., a custom element that responds to changes in its attributes.
- **Polyfilling missing features** – watch for new elements that need to be enhanced.

```javascript
// Example: Auto‑scroll to bottom when new messages are added
const chatContainer = document.getElementById('chat');
const observer = new MutationObserver(() => {
    chatContainer.scrollTop = chatContainer.scrollHeight;
});
observer.observe(chatContainer, { childList: true, subtree: true });
```

### Performance Considerations

Mutation observers can impact performance if they observe large subtrees or many changes. Use them sparingly and disconnect when no longer needed.

---

## 24.8 Resize Observer

The Resize Observer API provides a way to observe changes to an element's size. Unlike the `resize` event on `window`, Resize Observer can watch individual elements and fires when the element's content rectangle changes (e.g., due to CSS resizing, window resize affecting it, or content changes).

### Creating a Resize Observer

```javascript
const observer = new ResizeObserver((entries) => {
    for (const entry of entries) {
        console.log('Element:', entry.target);
        console.log('Content rect:', entry.contentRect);
        // entry.contentRect: width, height, top, left, right, bottom
    }
});

// Start observing an element
const element = document.getElementById('resizable');
observer.observe(element);

// To stop observing
observer.unobserve(element);
// or
observer.disconnect();
```

### Use Cases

- **Responsive components** – adjust internal layout when the component's container changes size (e.g., a chart that needs to redraw).
- **Adaptive text** – change font size or truncate text when the container shrinks.
- **Layout debugging** – monitor size changes during development.

```javascript
// Example: Keep a textarea's height auto‑adjusting
const textarea = document.getElementById('auto-grow');
const observer = new ResizeObserver(() => {
    textarea.style.height = 'auto';
    textarea.style.height = textarea.scrollHeight + 'px';
});
observer.observe(textarea, { box: 'border-box' });
```

Note: The `box` option in `observe()` can be `'content-box'` (default) or `'border-box'` to specify which box model to watch.

### Browser Support

Resize Observer is well‑supported in modern browsers, but for older ones you may need a polyfill.

---

## Chapter Summary

In this chapter, you explored a wide range of browser APIs that are essential for modern frontend development:

- **Timers** – scheduling tasks with `setTimeout` and `setInterval`, and patterns like debouncing and throttling.
- **`window` object** – controlling the browser window, scrolling, and using simple dialogs.
- **`navigator` object** – obtaining browser and device information, checking online status, and using the Clipboard API.
- **`location` object** – manipulating the current URL, redirecting, and reloading.
- **Web Storage API** – persisting data locally with `localStorage` and `sessionStorage`, and synchronizing across tabs with storage events.
- **Intersection Observer** – efficiently detecting element visibility for lazy loading, infinite scroll, and scroll‑based animations.
- **Mutation Observer** – reacting to DOM changes (additions, removals, attribute changes).
- **Resize Observer** – responding to element size changes for adaptive components.

### Key Takeaways

- Timers are fundamental for debouncing, throttling, and scheduling. Always clear timers to avoid memory leaks.
- The `window` object is the global scope; be mindful of overriding its properties.
- Use feature detection instead of parsing `userAgent` when possible.
- The Clipboard API is asynchronous and requires user interaction for security.
- Web Storage is synchronous and limited to strings; use JSON for complex data.
- Observers (Intersection, Mutation, Resize) are performance‑friendly alternatives to polling and event‑heavy patterns.
- Always disconnect observers when they are no longer needed to free up resources.

### Practice Exercises

1. **Debounced Search** – Create an input field that fetches suggestions from an API. Use `debounce` to limit requests. Display the results below the input.

2. **Lazy Loading Images** – Build a page with many images. Use Intersection Observer to load images only when they enter the viewport (with a rootMargin of 200px). Show a placeholder until the real image loads.

3. **Cross‑Tab Synchronization** – Create two HTML pages that both display a counter stored in `localStorage`. When you increment the counter on one page, the other page should automatically update using the `storage` event.

4. **Resize Observer Dashboard** – Build a dashboard with resizable panels (using CSS `resize`). Use Resize Observer to log the new dimensions and update some internal text (e.g., "Panel width: X").

5. **Mutation Observer for Dynamic Content** – Create a page with a button that adds new list items. Use Mutation Observer to count the total number of items and display the count somewhere. Also, whenever an item is added, log the added text.

6. **Clipboard Copy** – Add a button next to a code block that copies the code to the clipboard using the Clipboard API. Show a success message.

7. **Infinite Scroll with Intersection Observer** – Implement an infinite scroll that loads more posts from a dummy API when a sentinel element becomes visible. Use a loading spinner.

---

## Coming Up Next

**Chapter 25: Error Handling and Debugging**

In the next chapter, you'll learn how to handle errors gracefully, throw custom errors, and debug JavaScript effectively using browser DevTools. You'll also explore common JavaScript pitfalls and defensive programming techniques.